In [2]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
import pickle
import time

In [4]:
# read metadata
path = "/media/yesindeed/WD5T/data/CheXpert/CheXpert-v1.0-small"

df = pd.read_csv(os.path.join(path, "train.csv"))

In [47]:
df["Patient"] = df["Path"].apply(lambda x: x[26:38])
df

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,Patient
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,patient00001
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,patient00002
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,patient00002
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,patient00002
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,patient00003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223409,CheXpert-v1.0-small/train/patient64537/study2/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,-1.0,0.0,1.0,NaN,NaN,NaN,patient64537
223410,CheXpert-v1.0-small/train/patient64537/study1/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.0,-1.0,NaN,-1.0,NaN,NaN,NaN,patient64537
223411,CheXpert-v1.0-small/train/patient64538/study1/...,Female,0,Frontal,AP,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,patient64538
223412,CheXpert-v1.0-small/train/patient64539/study1/...,Female,0,Frontal,AP,NaN,NaN,1.0,1.0,NaN,NaN,NaN,-1.0,1.0,0.0,NaN,NaN,NaN,0.0,patient64539


In [48]:
df_race = pd.read_excel(os.path.join(path, "CHEXPERT DEMO.xlsx"), sheet_name="CheXpert Demo 11-13")

df = df.merge(df_race, how="left", left_on="Patient", right_on="PATIENT")
df["Race"] = df["PRIMARY_RACE"]
df = df.loc[df["PATIENT"].notna() & df["Patient"].notna()].drop(
    columns=["Patient", "PATIENT", "ETHNICITY", "AGE_AT_CXR", "PRIMARY_RACE"]
)

df

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,...,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,GENDER,Race
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,Female,Other
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,...,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,Female,"White, non-Hispanic"
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,...,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,Female,"White, non-Hispanic"
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,...,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,Female,"White, non-Hispanic"
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Male,"White, non-Hispanic"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223406,CheXpert-v1.0-small/train/patient64535/study1/...,Male,60,Frontal,AP,NaN,NaN,NaN,1.0,NaN,...,-1.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,Male,Black or African American
223407,CheXpert-v1.0-small/train/patient64536/study2/...,Female,61,Frontal,AP,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Female,Other
223408,CheXpert-v1.0-small/train/patient64536/study1/...,Female,61,Frontal,AP,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,Female,Other
223409,CheXpert-v1.0-small/train/patient64537/study2/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,...,NaN,NaN,-1.0,0.0,1.0,NaN,NaN,NaN,Male,Black or African American


In [49]:
df = df[df["Sex"].isin(df["GENDER"])].drop(columns="GENDER")
df

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,Race
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,Other
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,"White, non-Hispanic"
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,"White, non-Hispanic"
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,"White, non-Hispanic"
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,"White, non-Hispanic"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223406,CheXpert-v1.0-small/train/patient64535/study1/...,Male,60,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,Black or African American
223407,CheXpert-v1.0-small/train/patient64536/study2/...,Female,61,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Other
223408,CheXpert-v1.0-small/train/patient64536/study1/...,Female,61,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,Other
223409,CheXpert-v1.0-small/train/patient64537/study2/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,-1.0,0.0,1.0,NaN,NaN,NaN,Black or African American


In [50]:
# remove age/sex/race == null
df = df[~df["Sex"].isnull()]
df = df[~df["Age"].isnull()]
df = df[~df["Race"].isnull()]
df

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,Race
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,Other
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,"White, non-Hispanic"
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,"White, non-Hispanic"
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,"White, non-Hispanic"
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,"White, non-Hispanic"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223406,CheXpert-v1.0-small/train/patient64535/study1/...,Male,60,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,Black or African American
223407,CheXpert-v1.0-small/train/patient64536/study2/...,Female,61,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Other
223408,CheXpert-v1.0-small/train/patient64536/study1/...,Female,61,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,Other
223409,CheXpert-v1.0-small/train/patient64537/study2/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,-1.0,0.0,1.0,NaN,NaN,NaN,Black or African American


In [51]:
df["Race"].value_counts()

Race
White                                        102402
Other                                         28095
White, non-Hispanic                           22154
Asian                                         20434
Unknown                                       15186
Black or African American                      9909
Race and Ethnicity Unknown                     8716
Other, Hispanic                                3621
Native Hawaiian or Other Pacific Islander      2809
Asian, non-Hispanic                            2793
Black, non-Hispanic                            2000
White, Hispanic                                 922
Other, non-Hispanic                             566
American Indian or Alaska Native                457
Patient Refused                                 405
Pacific Islander, non-Hispanic                  337
Native American, non-Hispanic                    55
Black, Hispanic                                  52
Asian, Hispanic                                  37
Native 

In [52]:
mask = df["Race"].str.contains("White", na=False)
df.loc[mask, "Race"] = "White"
df.loc[~mask, "Race"] = "Non-White"

df["No Finding"].replace(-1, 0, inplace=True)
df["No Finding"].fillna(0, inplace=True)

In [53]:
df["Race"].value_counts()

Race
White        125491
Non-White     95515
Name: count, dtype: int64

In [54]:
df["Sex"].value_counts()

Sex
Male      131155
Female     89851
Name: count, dtype: int64

In [55]:
df["No Finding"].value_counts()

No Finding
0.0    198856
1.0     22150
Name: count, dtype: int64

In [56]:
df.to_csv(os.path.join(path, "test.csv"), index=False)

In [59]:
# sex

tem_train = pd.read_csv("/home/yesindeed/Documents/Dropbox/CXP-split/new_train.csv")
tem_test = pd.read_csv("/home/yesindeed/Documents/Dropbox/CXP-split/new_test.csv")

df = pd.read_csv(os.path.join(path, "all.csv"))
df_train = df.loc[df["Path"].isin(tem_train["Path"])].reset_index(drop=True)
df_test = df.loc[df["Path"].isin(tem_test["Path"])].reset_index(drop=True)

# assert len(df_train) == len(tem_train), f"{len(df_train)}, {len(tem_train)}"

df_train.to_csv(os.path.join(path, "train.csv"), index=False)
df_test.to_csv(os.path.join(path, "test.csv"), index=False)

In [13]:
# age

tem_test = pd.read_csv("/home/yesindeed/Documents/Dropbox/cxp_test_age.csv")

df = pd.read_csv(os.path.join(path, "all.csv"))
df_test = df.loc[df["Path"].isin(tem_test["Path"])].reset_index(drop=True)

# assert len(df_train) == len(tem_train), f"{len(df_train)}, {len(tem_train)}"

print(len(df_test))
df_test.to_csv(os.path.join(path, "test_age.csv"), index=False)

tem_test["Age_binary"].value_counts()

33757


Age_binary
0    17081
1    17081
Name: count, dtype: int64

In [10]:
df_test["Age"].value_counts()

Age
90    1128
57     797
58     796
61     753
56     750
      ... 
20     199
21     194
26     187
19     182
18      99
Name: count, Length: 73, dtype: int64

In [6]:
# race

tem_test = pd.read_csv("/home/yesindeed/Documents/Dropbox/cxp_test_race.csv")

df = pd.read_csv(os.path.join(path, "all.csv"))
df_test = df.loc[df["Path"].isin(tem_test["Path"])].reset_index(drop=True)

# assert len(df_train) == len(tem_train), f"{len(df_train)}, {len(tem_train)}"

df_test.to_csv(os.path.join(path, "test_race.csv"), index=False)